In [31]:
import pymysql
import pandas as pd
import classified_names
from geopy.geocoders import ArcGIS
from geopy.exc import GeocoderTimedOut
import json

In [2]:
#Dictionary for gender mapping
name_gender = classified_names.output
name_gender['Ararat'] = 'Male'

In [3]:
#Connecting to database
user = 'Copa'
password = 'COPA888'
host = 'localhost'
port = 3306
database = 'spyur'
connection = pymysql.connect(user=user, password=password, host=host, port=port, database=database)

In [ ]:
#Reading sql tables
company_t = "select * from company_table"
exec_t = 'select * from executive_table'
act_t = 'select * from activity_table'
prod_t = 'select * from product_table'
company_table = pd.read_sql(company_t, connection)
executive_table = pd.read_sql(exec_t, connection)
activity_table = pd.read_sql(act_t, connection)
product_table = pd.read_sql(prod_t, connection)

In [35]:
unique_activities = set(activity_table['activity_name'])
activities = list(unique_activities)
filename = 'activities.json'
with open(filename, 'w') as file:
    json.dump(activities, file, indent=4)

In [6]:
#Keeping only companies with one executive
executive_counts = executive_table.groupby('url_id').size()
single_executive_companies = executive_counts[executive_counts == 1]
executive_table = executive_table[executive_table['url_id'].isin(single_executive_companies.index)]

In [ ]:
#Splitting executive rows into full name and role columns
executive_table['executive'] = executive_table['executive'].astype('str')
executive_table['role'] = executive_table['executive'].str.split(',', n=1).str[1].str.strip()
executive_table['full_name'] = executive_table['executive'].str.split(',', n=1).str[0].str.strip()

In [8]:
#Function to extract first names for mapping
names = executive_table['full_name'].tolist()
def extract_names(name_lists, suffixes=["yan", "ian", "ts", "unc", "jan", "tz", "ians"]):
    last_names = []
    middle_names = []
    first_names = []

    for full_name in name_lists:
        if full_name:
            name_parts = full_name.split()
            current_first_names = []

            for i, part in enumerate(name_parts):
                if part.endswith('.'):
                    middle_names.append(part)
                elif any(part.endswith(suffix) for suffix in suffixes):
                    last_names.append(part)
                else:
                    current_first_names.append(part)

            first_names.append(' '.join(current_first_names))

    return last_names, middle_names, first_names

last_names, middle_names, first_names = extract_names(names)

In [ ]:
#Adding first_names column and adding gender by mapping
executive_table['first_name'] = [name.split()[0] for name in first_names]
executive_table['gender'] = executive_table['first_name'].map(name_gender)

In [20]:
activity_table

,url_id,activity_name_id,url,activity_name,act_first_level_cluster,act_second_level_cluster
0,00f23078c33a2cc78c905940ad0ba76a,0eec5d24c9a7262e790d9c77cc3e0241,https://www.spyur.am/en/companies/express-cour...,INTERNATIONAL CARGO TRANSPORTATION,COMPANIES AND NON-PROFIT ORGANIZATIONS,TRANSPORT
1,00f23078c33a2cc78c905940ad0ba76a,8c1979612e7f71a66508c244ad1bf387,https://www.spyur.am/en/companies/express-cour...,"DELIVERY OF LETTERS, DOCUMENTS, PARCELS, PACKA...",COMPANIES AND NON-PROFIT ORGANIZATIONS,TRANSPORT
2,03010d7c0383ee13828dd143210f7cf9,4523cff8e9027ea4f7a44f9990c57da5,https://www.spyur.am/en/companies/time-to-prin...,ADVERTISING AGENCY / ADVERTISING COMPANY,COMPANIES AND NON-PROFIT ORGANIZATIONS,"INFORMATION, MARKETING, ADVERTISING, PR"
3,03010d7c0383ee13828dd143210f7cf9,45a55e2441825e13ead2d235ee26e8ff,https://www.spyur.am/en/companies/time-to-prin...,PRINTING ON VARIOUS PRODUCTS AND MATERIALS,COMPANIES AND NON-PROFIT ORGANIZATIONS,PUBLISHING AND PRINTING WORK
4,03010d7c0383ee13828dd143210f7cf9,f28814552b348e7410f400b91a63a36d,https://www.spyur.am/en/companies/time-to-prin...,PRINTING HOUSE,COMPANIES AND NON-PROFIT ORGANIZATIONS,PUBLISHING AND PRINTING WORK
...,...,...,...,...,...,...
434,fbfeb236ebf6041a37174db9518b3f0e,fe9585947cadcef931bf603aaf4951dc,https://www.spyur.am/en/companies/s-design-stu...,DESIGN COMPANY,COMPANIES AND NON-PROFIT ORGANIZATIONS,VARIOUS SERVICES
435,fd937ddfdc92f264105d5e936bec7b57,16cd1a31ed10f7c10ea45f98fb2eaccc,https://www.spyur.am/en/companies/globus-tours...,OUTGOING TOURISM,COMPANIES AND NON-PROFIT ORGANIZATIONS,"TOURISM, RECREATION AND RELATED SERVICES"
436,fd937ddfdc92f264105d5e936bec7b57,1813269baa0ccd62c8ecc0825b5f2cc4,https://www.spyur.am/en/companies/globus-tours...,AIRLINE TICKETS (AIR TICKETS),COMPANIES AND NON-PROFIT ORGANIZATIONS,"TOURISM, RECREATION AND RELATED SERVICES"
437,fedfa400e6059633653b84c549e0757f,0eec5d24c9a7262e790d9c77cc3e0241,https://www.spyur.am/en/companies/kavkaz-trans...,INTERNATIONAL CARGO TRANSPORTATION,COMPANIES AND NON-PROFIT ORGANIZATIONS,TRANSPORT


In [11]:
executive_table

,url_id,url,executive,executive_id,role,full_name,first_name,gender
0,00f23078c33a2cc78c905940ad0ba76a,https://www.spyur.am/en/companies/express-cour...,"Arthur Avetisyan, Director",2f8b94292c685320d3aed16c2a9392c8,Director,Arthur Avetisyan,Arthur,Male
1,03f5b10e25bfb37f9eb461c0838e67e2,https://www.spyur.am/en/companies/iservice/47536,"Hrant A. Manvelyan, Director",c05a2c715ead062d45a216fdd033dd89,Director,Hrant A. Manvelyan,Hrant,Male
2,05e193da20f9f709f267b2aaf32f6b48,https://www.spyur.am/en/companies/sparsis-mark...,"Julietta Avetisyan, Director",d4bfb0b88772850564d474787ea5bbf3,Director,Julietta Avetisyan,Julietta,Female
3,0870a3aa7aba131d33dee1a020f2b62a,https://www.spyur.am/en/companies/ecomotors-am...,"Armen Abrahamyan, Director",8993c5057ecf05aebeb05d5421fd88a8,Director,Armen Abrahamyan,Armen,Male
4,09fae0a811a4c689027565b8e85db741,https://www.spyur.am/en/companies/alfa-pharm-p...,"Michael V. Matsakyan, General Director",e9bbffba20e3a783db681ea2439573fc,General Director,Michael V. Matsakyan,Michael,Male
...,...,...,...,...,...,...,...,...
92,f6ebd8fe1131616cef7cbf14ca61e92d,https://www.spyur.am/en/companies/cleaning-cen...,"Anush Manvelyan, Director",651e8e3bc7ebb7a607d5490f5f00b816,Director,Anush Manvelyan,Anush,Female
93,f950b50a83637fc4c7a3103ff60117d3,https://www.spyur.am/en/companies/arta-tour-tr...,"Armen A. Arakelyan, Founder",487a66e37b60a23f95cece2ebe2b0289,Founder,Armen A. Arakelyan,Armen,Male
94,fbfeb236ebf6041a37174db9518b3f0e,https://www.spyur.am/en/companies/s-design-stu...,"Suren Khachatryan, Director",481a26139528ac19b81a3288c95c1778,Director,Suren Khachatryan,Suren,Male
95,fd937ddfdc92f264105d5e936bec7b57,https://www.spyur.am/en/companies/globus-tours...,"Arman Danielyan, Director",e6530714e73d4e2a4fb44a5238b10f48,Director,Arman Danielyan,Arman,Male


In [12]:
# Function to geocode addresses and return (latitude, longitude)
def geocode_address(address):
    geolocator = ArcGIS(timeout=10)
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")
        return None, None

In [13]:
# Applying geocoding to each row in the DataFrame and create 'Location' column
locations = []
for index, row in company_table.iterrows():
    address = row['address']
    lat, lon = geocode_address(address)
    locations.append((lat, lon))
    print(f'Added location {lat}, {lon}')
print('Geocoding is comleted')

Added location 40.293084855, 44.940221311
Added location 40.18329, 44.51111
Added location 40.293084855, 44.940221311
Added location 40.293084855, 44.940221311
Added location 40.140927014938, 44.46664401538
Added location 40.194052984097, 44.487114969868
Added location 40.171842994248, 44.511048991471
Added location 40.293084855, 44.940221311
Added location 45.0712639, 7.6848003
Added location 40.167231983583, 44.522872001176
Added location 40.1889, 44.52087
Added location 40.293084855, 44.940221311
Added location 40.173966, 44.502747
Added location 40.293084855, 44.940221311
Added location 40.293084855, 44.940221311
Added location 40.153315006827, 44.479994962215
Added location 40.177916982294, 44.509073963626
Added location 40.171136986544, 44.521081039925
Added location 8.37456, -72.59442
Added location 40.293084855, 44.940221311
Added location 40.156297404404, 44.500004921142
Added location 40.197192006836, 44.492922958216
Added location 40.18329, 44.51111
Added location 53.0246519

In [15]:
#Adding location column
company_table['location'] = locations

In [16]:
company_table

,url_id,company_name,address,number_of_employees,form_of_ownership,year_established,date_of_information_update,url,location
0,00f23078c33a2cc78c905940ad0ba76a,EXPRESS COURIER Limited Liability Company (LLC),"Armenia, 0015, YerevanParonyan St., 1/18 Building",Up to 15,Non-governmental,2019,12.07.2024,https://www.spyur.am/en/companies/express-cour...,"(40.293084855, 44.940221311)"
1,03f5b10e25bfb37f9eb461c0838e67e2,ISERVICE Limited Liability Company (LLC),"Armenia, 0025, YerevanCharentsi St., 66 Building",Up to 15,Non-governmental,2015,06.05.2024,https://www.spyur.am/en/companies/iservice/47536,"(40.18329, 44.51111)"
2,05e193da20f9f709f267b2aaf32f6b48,SPARSIS MARKETING COMPANY,"Armenia, 0051, YerevanVratsakan St. 4th Lane, ...",Up to 15,Non-governmental,2019,04.07.2024,https://www.spyur.am/en/companies/sparsis-mark...,"(40.293084855, 44.940221311)"
3,0870a3aa7aba131d33dee1a020f2b62a,ECOMOTORS AM,"Armenia, 0054, YerevanMelkumovi St., 88 Building",N/A,Non-governmental,2022,13.06.2024,https://www.spyur.am/en/companies/ecomotors-am...,"(40.293084855, 44.940221311)"
4,09fae0a811a4c689027565b8e85db741,ALFA-PHARM PHARMACY CHAIN,"Armenia, 0043, YerevanShiraki St., 1/68 Building",From 251 to 1000,Non-governmental,1996,26.01.2024,https://www.spyur.am/en/companies/alfa-pharm-p...,"(40.140927014938, 44.46664401538)"
...,...,...,...,...,...,...,...,...,...
90,f6ebd8fe1131616cef7cbf14ca61e92d,CLEANING CENTRE CLEANING SERVICE (CLEANING CEN...,"Armenia, 0033, YerevanHakob Hakobyan St., 2 Bu...",Up to 15,Non-governmental,2015,17.05.2024,https://www.spyur.am/en/companies/cleaning-cen...,"(40.201163981204, 44.491870023539)"
91,f950b50a83637fc4c7a3103ff60117d3,ARTA TOUR TRAVEL AGENCY,"Armenia, 0002, YerevanTumanyan St., 38 Buildin...",N/A,Non-governmental,N/A,16.07.2024,https://www.spyur.am/en/companies/arta-tour-tr...,"(40.293084855, 44.940221311)"
92,fbfeb236ebf6041a37174db9518b3f0e,S DESIGN STUDIO OUTDOOR ADVERTISING AND DESIGN...,"Armenia, 0033, YerevanGyulbenkyan St., 31 Buil...",Up to 15,Non-governmental,2007,04.07.2024,https://www.spyur.am/en/companies/s-design-stu...,"(40.200120979082, 44.495722010963)"
93,fd937ddfdc92f264105d5e936bec7b57,GLOBUS TOURS TRAVEL AGENCY Limited Liability C...,"Armenia, 0070, YerevanAlek Manukyan St., 9 Bui...",Up to 15,Non-governmental,2010,05.06.2024,https://www.spyur.am/en/companies/globus-tours...,"(40.293084855, 44.940221311)"


In [17]:
#converting year_extablished column type to int and filling nans
company_table['year_established'] = pd.to_numeric(company_table['year_established'], errors='coerce')
company_table['year_established'] = company_table['year_established'].fillna(1990).astype('int')

In [18]:
final_df1 = pd.merge(company_table, executive_table, on='url_id', how='left')

In [21]:
final_df = pd.merge(final_df1, activity_table, on='url_id', how='left')

In [22]:
#Saving Dataframe as pkl file
final_df.to_pickle('final_df.pkl')